<a href="https://colab.research.google.com/github/ngolla/video-captioning/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
from google.colab import drive
from pathlib import Path
import numpy as np

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_path=Path('/content/drive/My Drive')
list(drive_path.glob('*'))

[PosixPath('/content/drive/My Drive/Colab Notebooks'),
 PosixPath('/content/drive/My Drive/YouTubeClips.tar'),
 PosixPath('/content/drive/My Drive/video_corpus.csv'),
 PosixPath('/content/drive/My Drive/.ipynb_checkpoints'),
 PosixPath('/content/drive/My Drive/VideoArrays'),
 PosixPath('/content/drive/My Drive/model1.h5'),
 PosixPath('/content/drive/My Drive/splitdata'),
 PosixPath('/content/drive/My Drive/splitset'),
 PosixPath('/content/drive/My Drive/test.dat'),
 PosixPath('/content/drive/My Drive/tokens_word_index.txt'),
 PosixPath('/content/drive/My Drive/tokens_index_word.txt')]

In [3]:
split_path=drive_path.joinpath('splitset')
list(split_path.glob('*'))

[PosixPath('/content/drive/My Drive/splitset/train.dat'),
 PosixPath('/content/drive/My Drive/splitset/test.dat'),
 PosixPath('/content/drive/My Drive/splitset/tokens_word_index.txt'),
 PosixPath('/content/drive/My Drive/splitset/tokens_index_word.txt')]

In [4]:
train_file=split_path.joinpath('train.dat')
val_file=split_path.joinpath('test.dat')

In [5]:
(vid_test, cap_test)=pickle.load(open( val_file, "rb" ))
(vid_train, cap_train)=pickle.load(open( train_file, "rb" ))

In [8]:
print(len(vid_train), vid_train[0].shape)
print(len(vid_test), vid_test[0].shape)

8000 (80, 4096)
2000 (80, 4096)


In [6]:
vid_train=np.array(vid_train)
cap_train=np.array(cap_train)

In [ ]:
#vid_test=np.array(vid_test)
#cap_test=np.array(cap_test)

In [7]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# returns train, inference_encoder and inference_decoder models
def basic_enc_dec(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    # return all models
    return model, encoder_model, decoder_model

In [11]:
#split_path.joinpath('tokens_word_index.txt')
token=pickle.load(open(split_path.joinpath('tokens_word_index.txt'),"rb"))
len(token.items())

3434

In [17]:
cap_train.shape[1]
vid_train.shape[2]
cap_train.shape[1]

(8000, 35)

In [12]:
vocab_size=len(token.items())+1
#dim_embedding=64
maxlen=cap_train.shape[1]

In [13]:
model, enc, dec = basic_enc_dec(vid_train.shape[2], vocab_size, maxlen)

In [16]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 4096)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 3435)] 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 35), (None,  578480      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 35), ( 485940      input_2[0][0]                    
                                                                 lstm[0][1]            

In [19]:
x2 = np.hstack([np.zeros((cap_train.shape[0], 1)), np.array(cap_train)])
x2 = x2[:, :-1]

In [ ]:
#Convert to (no of captions)x(no of words)x(vocab size)
from keras.utils.np_utils import to_categorical   

#x2_in = to_categorical(x2, num_classes = vocab_size)
x2 = to_categorical(x2, num_classes = vocab_size)
outputs = to_categorical(cap_train, num_classes = vocab_size)
#print(x2_in.shape, outputs.shape)
print(x2.shape, outputs.shape)

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=7e-4), loss='categorical_crossentropy')
history=model.fit([vid, x2_in], outputs, epochs = 1)

In [ ]:
history.history

{'loss': [4.78618049621582]}